# CModel Analysis and Testing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sdfec_cmodel

In [2]:
with open("data/data_2.vcd", "r") as f:
    lines = f.read().split("\n")
    data = [int(v[1:], 2) for v in lines if len(v) > 0]

In [3]:
block = 8991712455
din_beats = 372
blocks = blocks = len(data) // din_beats
decoder = 4

### Simulation Setup

In [4]:
xoro = sdfec_cmodel.xoroshiro128plus(0xC3D3EE97)

for i in range(decoder*12):
    xoro.jump()

xoro.forward(block*din_beats)

for i in range(12):
    print(f'        (X"{xoro.s[1]:016x}", X"{xoro.s[0]:016x}"),')
    xoro.jump()

        (X"8df312f829d30e32", X"4f025507420e2260"),
        (X"f9a80ef3c7df9fe4", X"f935315d8c504088"),
        (X"08be1c6c77dd6251", X"feffa52e363444a4"),
        (X"1242605bee1d584a", X"7ed5eb6997cf9284"),
        (X"fd931074f18e941c", X"36ad97a948e0b8cf"),
        (X"f8a1c06ac2bc5daf", X"f5bd6904b06edb4b"),
        (X"3f5d35e21dfb4ae2", X"70baa462df9c2733"),
        (X"3f3cc90eb8f3ff90", X"6bdcf5e1997d04ad"),
        (X"4c2766c865e1b6bf", X"d913d0ff4f7b5ee1"),
        (X"683bdbc4a54e8372", X"3a41df1fc1c8e524"),
        (X"cad2b0e6dd89fa11", X"0ad6fa885ef2dcf7"),
        (X"803440e11c347841", X"37a0aaab4efaf00e"),


In [5]:
def split_vector(v, count, l):
    out = []
    mask = (1 << l)-1
    
    for i in range(count):
        out.append(v & mask)
        v >>= l
    
    return np.array(out, dtype=np.int8).astype(float) / 4.0
    

In [6]:
din_sim = np.array([split_vector(v, 16, 8) for v in data])
din_sim = din_sim.reshape(din_sim.size)

In [7]:
model = sdfec_cmodel.sdfec_core("sdfec", "docsis_31_medium.yml")

sdfec NOTE: Initializing LDPC code definitions from file...
sdfec NOTE: Code number: 0 -> no_id 


In [8]:
ber = sdfec_cmodel.ldpc_ber_tester(decoder, model)

In [9]:
ber.snr_scale = 4
ber.snr = 0

In [10]:
din_mod_raw,status,_,dout = ber.simulate_block(block, max_iterations=32, term_on_no_change=False)

In [11]:
din_mod = np.array(din_mod_raw)

In [12]:
din_sim[:128]

array([ 4.  , -4.  , -4.25, -5.25, -4.75, -1.  , -1.25, -3.25, -3.75,
       -6.  , -3.25, -3.75, -1.5 ,  2.75,  2.5 ,  2.25, -1.  , -7.75,
       -7.75,  1.  , -2.5 , -7.75,  1.5 , -7.75,  1.5 , -5.75,  0.25,
       -3.  , -0.25, -7.75, -5.75, -3.75, -0.75, -1.  , -6.25, -7.  ,
       -6.75, -3.25, -2.25, -7.5 ,  0.  , -7.75, -5.75, -3.75, -5.75,
       -2.5 , -7.75,  2.  ,  4.5 , -7.75, -6.  ,  2.  ,  1.25, -7.75,
       -0.5 , -1.  ,  4.25,  2.  , -0.25, -3.5 , -4.5 , -7.75, -2.75,
       -7.5 , -6.5 ,  0.  , -4.25, -2.75, -5.25, -3.25,  2.  ,  1.  ,
       -7.75, -7.75, -7.75, -5.5 , -5.75, -4.75, -7.  , -3.25, -5.75,
       -4.5 , -4.75, -4.  , -2.  , -7.75,  0.75, -7.75, -7.75, -7.75,
        0.75, -7.75, -3.  ,  2.5 , -4.75, -7.75, -7.75, -7.75, -6.75,
       -3.5 , -6.5 , -3.5 ,  0.5 , -3.  , -3.5 , -1.25, -5.25,  5.  ,
       -4.  ,  4.  , -7.75, -7.75,  1.25,  1.5 , -6.25, -1.75, -7.25,
       -6.  , -7.  , -1.5 , -7.25, -7.25, -4.25, -2.  , -1.75, -7.75,
       -3.5 ,  4.  ]

In [13]:
din_mod[:128]

array([ 4.  , -4.  , -4.25, -5.25, -4.75, -1.  , -1.25, -3.25, -3.75,
       -6.  , -3.25, -3.75, -1.5 ,  2.75,  2.5 ,  2.25, -1.  , -7.75,
       -7.75,  1.  , -2.5 , -7.75,  1.5 , -7.75,  1.5 , -5.75,  0.25,
       -3.  , -0.25, -7.75, -5.75, -3.75, -0.75, -1.  , -6.25, -7.  ,
       -6.75, -3.25, -2.25, -7.5 ,  0.  , -7.75, -5.75, -3.75, -5.75,
       -2.5 , -7.75,  2.  ,  4.5 , -7.75, -6.  ,  2.  ,  1.25, -7.75,
       -0.5 , -1.  ,  4.25,  2.  , -0.25, -3.5 , -4.5 , -7.75, -2.75,
       -7.5 , -6.5 ,  0.  , -4.25, -2.75, -5.25, -3.25,  2.  ,  1.  ,
       -7.75, -7.75, -7.75, -5.5 , -5.75, -4.75, -7.  , -3.25, -5.75,
       -4.5 , -4.75, -4.  , -2.  , -7.75,  0.75, -7.75, -7.75, -7.75,
        0.75, -7.75, -3.  ,  2.5 , -4.75, -7.75, -7.75, -7.75, -6.75,
       -3.5 , -6.5 , -3.5 ,  0.5 , -3.  , -3.5 , -1.25, -5.25,  5.  ,
       -4.  ,  4.  , -7.75, -7.75,  1.25,  1.5 , -6.25, -1.75, -7.25,
       -6.  , -7.  , -1.5 , -7.25, -7.25, -4.25, -2.  , -1.75, -7.75,
       -3.5 ,  4.  ]

In [14]:
mismatches = np.where(din_sim[:5940] - din_mod)[0]

In [15]:
mismatches

array([], dtype=int64)

In [16]:
din_mod[mismatches]

array([], dtype=float64)

In [17]:
din_sim[mismatches]

array([], dtype=float64)

In [18]:
bad_idx = 5410
beat_idx = bad_idx // 16
bm_core_idx = (bad_idx % 16)//2

print(f"Beat: {beat_idx}, Box-Muller Core: {bm_core_idx}")

Beat: 338, Box-Muller Core: 1


In [19]:
rnd_vector = ber.get_rnd_vector(block, bm_core_idx)[beat_idx]
print(f'X"{rnd_vector[0]:012x}", X"{rnd_vector[1]:04x}", X"{rnd_vector[2]:08x}"')

X"b5e87ea0c7ec", X"0eb9", X"7fffeb38"


In [20]:
sdfec_cmodel.bitexact_gaussian(*rnd_vector)

(0.00048828125, 0.0009765625)

In [21]:
rnd_vector = (0xb5e87ea0c7ec, 0x0eb9, 0x7fffeb38)

In [22]:
sdfec_cmodel.gaussian(*rnd_vector)

(0.0005564478274236821, 0.0014723061496036947)

In [23]:
sdfec_cmodel.bitexact_gaussian(*rnd_vector)

(0.00048828125, 0.0009765625)

In [24]:
din_sim[bad_idx]

-4.0